In [1]:
import numpy as np
from tqdm import tqdm
import sys
from keras import models
from keras import layers
sys.path.append('utils/')

from keras.models import Model

Using TensorFlow backend.


In [2]:
def preprocess_input(x):
    x /= 255.0
    x -= 0.5
    x *= 2.0
    return x

In [3]:
data_dir = 'E:/Jupyter NB/10 class Classification/'

train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
#test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [4]:
from image_preprocessing_ver1 import ImageDataGenerator

data_generator = ImageDataGenerator(data_format='channels_last', preprocessing_function = preprocess_input)
#val_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)

validation_generator = data_generator.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)
'''
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)
'''

print(train_generator)

Found 2000 images belonging to 20 classes.
Found 400 images belonging to 20 classes.


In [5]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

model.summary()

model.load_weights('teacher_weights.hdf5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 128)      

In [6]:
# remove softmax
model.layers.pop()
model = Model(model.input, model.layers[-1].output)
# now model outputs logits

In [7]:
from tqdm import tqdm
import numpy as np

batches = 0
train_logits = {}

for x_batch, _, name_batch in tqdm(train_generator):
    #print(np.shape(x_batch))
    #print(name_batch)
    #print(len(x_batch))
    #print(x_batch)
    
    batch_logits = model.predict_on_batch(x_batch)
    
    '''
    # Checking
    
    print(np.shape(batch_logits))
    #print(batch_logits)
    for i, n in enumerate(name_batch):
        print(batch_logits[i])
        print(n)
    '''
    
    for i, n in enumerate(name_batch):
        #print(i)
        #print(n)
        #print(train_logits)
        #print(batch_logits[i])
        train_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 100: # 2000/20
        break

99it [00:29,  6.16it/s]

In [8]:
batches = 0
val_logits = {}

for x_batch, _, name_batch in tqdm(validation_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
        val_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 20: # 400/20
        break


0it [00:00, ?it/s]
1it [00:00,  3.23it/s]
2it [00:00,  3.69it/s]
3it [00:00,  4.17it/s]
4it [00:00,  4.49it/s]
5it [00:01,  4.76it/s]
6it [00:01,  5.01it/s]
7it [00:01,  5.17it/s]
8it [00:01,  5.22it/s]
9it [00:01,  5.04it/s]
10it [00:01,  5.24it/s]
11it [00:02,  5.33it/s]
12it [00:02,  5.39it/s]
13it [00:02,  5.55it/s]
14it [00:02,  5.24it/s]
15it [00:02,  5.16it/s]
16it [00:03,  5.29it/s]
17it [00:03,  5.31it/s]
18it [00:03,  5.52it/s]
19it [00:03,  5.66it/s]

In [9]:
np.save(data_dir + 'train_logits.npy', train_logits)
np.save(data_dir + 'val_logits.npy', val_logits)